<a href="https://colab.research.google.com/github/martindisley/DeepFake-Doppelganger/blob/main/DeepFake_Doppleganger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Fake Dopplegänger**
### *Generate an avatar that preserves your likeness but hides your biometric data from facial recognition algorithms using StyleGAN and Avatarify.*

The first part of this notebook generates a StyleGAN dopplegänger. The second part allows you to offload the computation required to use it with Avatarify to this notebook while you run application locally.

The [Avatarify](https://github.com/alievk/avatarify.git) installation requires some familiarity with the command line and Anaconda environment set up. You will also need an [ngrok](https://dashboard.ngrok.com/get-started/setup) account in order to communicate between the app, running locally, and the computation running in the cloud.

Repos used:
1.   https://github.com/PDillis/stylegan2-ada
2.   https://github.com/ageitgey/face_recognition.git
3.   https://github.com/alievk/avatarify.git

To run the code in the notebook cells use the play button.

Currently the notebook will **only work with png files**.

---

## Upload portrait

In [ ]:
#@title Upload an image { display-mode: "form" }
!mkdir /content/raw_images /content/aligned_images
from google.colab import files
%cd /content/raw_images 
files.upload()

## Create Dopplegänger
The code to create the doppelgänger is hidden in the cells below. Use the play button to run the code or expand the cells to read the code and run each individually.

The code will take about 10 minutes to run.

The following is going on under the hood:
1.   The uploaded input image is cropped, aligned and resized and then projected into the FFHQ StyleGAN network to find the closest match it can produce. At each step of the projection the progress image is saved.
2.   To find the nearest image that also fools a facial verification algorithm, the script traverses back through the steps of the projection, comparing the face in each step to the input image. The script stops when the facial verification algorithm returns a negative result.



Import libraries 

In [ ]:
%cd /content/
!git clone https://github.com/PDillis/stylegan2-ada -q
%cd stylegan2-ada/
!mv /content/stylegan2-ada/ffhq_dataset/ /content/stylegan2-ada/utils/ffhq_dataset/

In [ ]:
import os
%cd /content/raw_images

if os.path.exists("/content/raw_images/input.ipynb_checkpoints"):
  os.rmdir("/content/raw_images/input.ipynb_checkpoints")

for files in os.listdir('/content/raw_images'):
  print(files)
  ext = files.split('.')[-1]
  os.rename('/content/raw_images/'+files, '/content/raw_images/input.{}'.format(ext))
%cd ../stylegan2-ada/

Align and crop images


In [ ]:
!python /content/stylegan2-ada/utils/align_faces.py /content/raw_images /content/aligned_images

Generate doppelgänger 

In [ ]:
%tensorflow_version 1.x
!python /content/stylegan2-ada/projector.py --outdir=/content/ --target=/content/aligned_images/input_01.png --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl --save-every-step --num-steps=300

Compare faces for similarity

In [ ]:
# Import prerequisite modules
%cd /content/
!git clone https://github.com/ageitgey/face_recognition.git -q
%cd /content/face_recognition/
!pip install git+https://github.com/ageitgey/face_recognition_models -q

Compare faces

In [ ]:
import face_recognition
import shutil

filenames = os.listdir('/content/steps')
sorted_filenames = sorted(filenames, key=lambda x: int(x.split('_')[1].split('.')[0]))
for steps in reversed(sorted_filenames):
  if steps.split('.')[-1] == 'jpg':
    known_image = face_recognition.load_image_file("/content/target.png")
    unknown_image = face_recognition.load_image_file("/content/steps/{}".format(steps))
    known_encoding = face_recognition.face_encodings(known_image)[0]
    unknown_encoding = face_recognition.face_encodings(unknown_image)[0]
    result = face_recognition.compare_faces([known_encoding], unknown_encoding)
    print(steps, result[0])
    if result == [False]:
      shutil.copy2("/content/steps/{}".format(steps), '/content/minimal_diff_doppelganger.png')
      break

## Download Doppelgänger
Once you've downloaded your image add it to the *avatars* sub directory within Avatarify to use it with the application.

In [ ]:
#@title Download image
from google.colab import files
files.download('/content/minimal_diff_doppelganger.png')

---

#Colab Rendering Server

The cells below run an Avatarify rendering server, allowing you to run Avatarify on your computer **without GPU**:

1. When the following cells are executed, a rendering server starts listening for incoming requests from the client (your local machine).
1. Starting the client on your computer connects it to this notebook and starts sending requests
1. This notebooks then receives the requests from your computer, renders avatar images and sends them back

**Start the server**

Run the cells below (Shift+Enter) sequentially and pay attention to the hints and instructions included in this notebook.

At the end you will get a command for running the client on your computer.

**Start the client**

Make sure you have installed the latest version of Avatarify on your computer. Refer to the [README](https://github.com/alievk/avatarify#install) for the instructions.

When it's ready execute this notebook and get the command for running the client on your computer.


### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [ ]:
#@title Input ngrok token { display-mode: "form" }
import ipywidgets as widgets
authtoken_input = widgets.Text(value='Paste your authtoken here')
display(authtoken_input)

## Run


Install Avatarify and ngrok

In [ ]:
%cd /content
!git clone https://github.com/alievk/avatarify.git
%cd avatarify
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy pyyaml==5.1
!scripts/download_data.sh
# Download ngrok
!scripts/get_ngrok.sh

In [ ]:
authtoken = authtoken_input.value

In [ ]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [ ]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

In [ ]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

In [ ]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

In [ ]:
# Set your region here in quotes
region = "eu"

In [ ]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [ ]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [ ]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

## Start the client
When you run the cell below it will print a command. Run this command on your computer:

1. Open a terminal (in Windows open `Anaconda Prompt`);
2. Change working directory to the `avatarify` directory:</br>
* Windows (change `C:\path\to\avatarify` to your path)</br>
`cd C:\path\to\avatarify`</br></br>
* Mac/Linux (change `/path/to/avatarify` to your path)</br>
`cd /path/to/avatarify`
3. Copy and paste to the command below and run it in your local terminal

In [ ]:
#@title Get terminal command
print('Copy-paste to the terminal the command below and run (press Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

#### Logs
If something doesn't work as expected, please run the cells below and include the logs in your report.

In [ ]:
#@title
!cat ./var/log/cam_fomm.log | head -100

In [ ]:
#@title
!cat ./var/log/recv_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/send_worker.log | tail -100